# End-To-End Example: Zomato Map

Let's import our zomato functions and then use folium to plot restaurants on a map.



In [1]:
import requests 
import pandas as pd
import folium
import import_ipynb
import zomato as zomato
from IPython.display import display, HTML
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

importing Jupyter notebook from zomato.ipynb


In [2]:
zomato_key = '5aee574594af0c43a13d8f5e81744b73'

In [3]:
def geocode(location):
    '''
    using geocoding here: https://nominatim.openstreetmap.org/search
    '''
    query_string = {'q' : location, 'format' :'json'}
    url = 'https://nominatim.openstreetmap.org/search'
    response = requests.get(url, params = query_string)
    data = response.json()
    if type(data) == list:
        return data[0]
    else:
        return data
    

In [5]:
display(HTML("<h1>Yummy.com Restaurants</h1>"))
interact_zomato=interact.options(manual=True, manual_name="Find Eateries!")
@interact_zomato(cuisine="", city="")
def main(cuisine,city):
    city_id = zomato.get_city_id(zomato_key, city)
    rests = zomato.get_restaurants(zomato_key, city_id, cuisine)
    rests_df = pd.json_normalize(rests)
    geo = geocode(city)
    MAPCITY = (geo['lat'], geo['lon'])
    rmap = folium.Map(location=MAPCITY, zoom_start=10, width=800, height=600)
    #rests_filter = rests_df[ rests_df['restaurant.cuisines'].str.find(cuisine.title()) >=0 ]
                    
    for row in rests_df.to_records():
        coords = (float(row['restaurant.location.latitude']),float(row['restaurant.location.longitude']))
        loc = f"<p><b>{row['restaurant.name']}</b></p>"
        loc += f"<p>({row['restaurant.cuisines']})</p>"
        marker = folium.Marker(location=coords, popup=loc,tooltip=loc)
        rmap.add_child(marker)
    display(rmap)

interactive(children=(Text(value='', description='cuisine'), Text(value='', description='city'), Button(descri…